## Introduction:

> Kickstarter is an American public-benefit corporation[2] based in Brooklyn, New York, that maintains a global crowdfunding platform focused on creativity and merchandising.[3] The company's stated mission is to "help bring creative projects to life".[4] Kickstarter has reportedly received more than $1.9 billion in pledges from 9.4 million backers to fund 257,000 creative projects, such as films, music, stage shows, comics, journalism, video games, technology and food-related projects.[5]
People who back Kickstarter projects are offered tangible rewards or experiences in exchange for their pledges.[6] This model traces its roots to subscription model of arts patronage, where artists would go directly to their audiences to fund their work.[7][Wikipedia](https://en.wikipedia.org/wiki/Kickstarter)

So, with the help of the crawlers of [webrobots](https://webrobots.io/kickstarter-datasets/), we got all the data from the run along 2017 kickstater projects and keep just those written in english and finished either as "successful" or "failed", and two columns:

- the one with the blurb or short description of the project [text]
- the one with the final state: "successful" if the project got the money goal in time or "failed" if don't [factor]

This kernel will deploy a linguistic model with embeddings and RNN which predict from the blurb/short description of the project if will be "successful" or "failed".

Another works, like this from [Sawhney et al](https://stanford.edu/~kartiks2/kickstarter.pdf), propose to include other variables of the project and the NLP part to get a 0.7 accuracy, but we'll use here just the NLP part.

## Getting and preprocessing data:

The first is to get the dataset which have been disposed as public [here](https://www.kaggle.com/oscarvilla/kickstarter-nlp) on Kaggle with pandas, and to map the categories to numeric ones: 1 if "successful" and 0 if "failed".

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("../input/df_text_eng.csv")
df["state"] = df["state"].map({'successful': 1, 'failed': 0})
df.head()

Here we need to pass from pandas dataframe to list of strings

In [ ]:
texts = list(df["blurb"].astype(str).str.split(",")) # Convert the column to string and to list... list of strings
labels = list(df["state"].astype(str).str.split(","))

Sanity checks of the texts and the labels

In [ ]:
texts[1:6]

In [ ]:
labels[1:6]

The preprocess needed here it's to convert the words sequences to a sequences of numbers, where each number represent a word. That's done with a tokenizer to which we've defined the more frequent words to take in account (20000).
Also, we have to define a fixed length for the sequences (25): truncating the larger and padding the shorters.

Lastly, we define how much of the sample will be used for training and how much for validation and test.

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

maxlen = 25
training_samples = 172410
validation_samples = 21551
test_samples = 21551
max_words = 20000

tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Padding and check dimensions.

In [ ]:
data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of labels tensor:', labels.shape)

Here we shuffle the data and create partitions

In [ ]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

In [ ]:
x_train = data[:training_samples]
y_train = labels[:training_samples]
k = training_samples + validation_samples
x_val = data[training_samples: k]
y_val = labels[training_samples: k]
l = training_samples + validation_samples + test_samples
x_test = data[k:l]
y_test = labels[k:l]

In [ ]:
print('Shape of x_train tensor:', x_train.shape)
print('Shape of x_val tensor:', x_val.shape)
print('Shape of x_test tensor:', x_test.shape)

## The model:

Finally we define the model, compile it and run for training

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM

embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(LSTM(100, recurrent_dropout = 0.3, dropout = 0.3))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])
history = model.fit(x_train, y_train,
epochs=3,
batch_size=64,
validation_data=(x_val, y_val))